# Pipeline 2
+ knowledge base
+ vector database
+ rag

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

In [3]:
from dotenv import load_dotenv
load_dotenv("/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/.env", override=True)

True

## Knowledge base
In this component we perform the loading of the knowledge base

In [ ]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/0.knowledge_base
!pip install -r requirements.txt

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/0.knowledge_base


In [ ]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files


In [ ]:
import rdflib
import json
import os

def knowledge_base():
  def parse_rdf(file_path):
      g = rdflib.Graph()
      try:
          g.parse(file_path, format='ttl')
      except rdflib.exceptions.ParserError as e:
          print(f"Errore di parsing: {e}")
      triples = [(str(s), str(p), str(o)) for s, p, o in g]
      return triples

  input_path = os.getenv("input_path")
  rdf_triples = parse_rdf(input_path)
  print(f'{len(rdf_triples)} triples extracted')
  with open(os.getenv("output_path"), 'w', encoding='utf-8') as f:
    json.dump(rdf_triples, f, ensure_ascii=False, indent=4)
if __name__ == '__main__':
  knowledge_base()

31310 triples extracted


In [ ]:
with open("/content/triples.json", "r") as file:
      triples = json.load(file)

In [ ]:
for triple in triples:
  print(triple)

['http://example.org/entities/MailDatabase', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://example.org/d3f/Database']
['http://example.org/entities/IMAPServer', 'http://example.org/d3f/reads', 'http://example.org/entities/MailDatabase']
['http://example.org/entities/EmailClient', 'http://example.org/d3f/SendsEmail', 'http://example.org/entities/SMTPServer']
['http://example.org/entities/EmailClient', 'http://www.w3.org/2000/01/rdf-schema#label', 'Email Client']
['http://example.org/entities/IMAPServer', 'http://www.w3.org/2000/01/rdf-schema#label', 'IMAP Server']
['http://example.org/entities/MailDatabase', 'http://www.w3.org/2000/01/rdf-schema#label', 'Mail Database']
['http://example.org/entities/SMTPServer', 'http://example.org/d3f/writes', 'http://example.org/entities/MailDatabase']
['http://example.org/entities/SMTPServer', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://example.org/d3f/MailServer']
['http://example.org/entities/IMAPServer', 'http://example

## Vector Database

In [14]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/1.vector_database
!pip install -r requirements.txt

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/1.vector_database
  Cloning https://github.com/pykeen/pykeen.git to /tmp/pip-req-build-mx3pryss
  Running command git clone --filter=blob:none --quiet https://github.com/pykeen/pykeen.git /tmp/pip-req-build-mx3pryss
  Resolved https://github.com/pykeen/pykeen.git to commit 8b4d4811bb70a9765f4c5dd45011cdd87749502a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00
  Created wheel for pykeen: filename=pykeen-1.11.1.dev0-py3-none-any.whl size=718682 sha256=abf09cc8facb1ace0040da491201592aabf5c32e386c4c7df8d0cefae8519e37
  Stored in directory: /tmp/pip-ephem-wheel-cache-3kwc_n1z/wheels/98/cc/ca/e65a97e466f82422e756

In [15]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files


In [37]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Configure, Property, DataType
import re
import os
import json

# Funzione per estrarre l'ultima parte dell'URL
def extract_name(url):
    return re.split(r'[#/]', url)[-1]

def vector_database():
    weaviate_url = os.getenv("WEAVIATE_URL")
    weaviate_api_key = os.getenv("WEAVIATE_API_KEY")

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={
            "X-OpenAI-Api-Key": os.environ["OPENAI_API_TOKEN"]
        }
    )

    print(client.is_ready())

    client.collections.create(
        "Triple",
        vectorizer_config=Configure.Vectorizer.text2vec_openai(),
        properties=[
            Property(name="triple", data_type=DataType.TEXT),
            Property(name="source", data_type=DataType.TEXT)
        ]
    )

    with open(os.getenv("output_path"), "r") as file:
        triples = json.load(file)

    for index, triple in enumerate(triples[:100], start=1):
        subject = extract_name(triple[0])
        predicate = extract_name(triple[1])
        object_ = extract_name(triple[2])
        source = subject + '--' + predicate + '--' + object_

        # Payload da caricare
        data_object = {
            "triple": source,
            "source": source
        }

        collection = client.collections.get("Triple")

        with collection.batch.dynamic() as batch:
            batch.add_object(properties=data_object)

        # Stampa lo stato di avanzamento
        print(f"Caricata tripla {index} di {len(triples)}: {data_object}")

if __name__ == '__main__':
    vector_database()


True
Caricata tripla 1 di 31310: {'triple': 'Dyna-Q--label--Dyna-Q', 'source': 'Dyna-Q--label--Dyna-Q'}
Caricata tripla 2 di 31310: {'triple': 'T1555.005--definition--Adversaries may acquire user credentials from third-party password managers.(Citation: ise Password Manager February 2019) Password managers are applications designed to store user credentials, normally in an encrypted database. Credentials are typically accessible after a user provides a master password that unlocks the database. After the database is unlocked, these credentials may be copied to memory. These databases can be stored as files on disk.(Citation: ise Password Manager February 2019)', 'source': 'T1555.005--definition--Adversaries may acquire user credentials from third-party password managers.(Citation: ise Password Manager February 2019) Password managers are applications designed to store user credentials, normally in an encrypted database. Credentials are typically accessible after a user provides a maste

## RAG

In [4]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/2.rag
!pip install -r requirements.txt

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/2.rag
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.2 MB/s e

In [17]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files


In [38]:
import os
from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import weaviate
from weaviate.classes.init import Auth
import textwrap

def rag():
    weaviate_url = os.getenv("WEAVIATE_URL")
    weaviate_api_key = os.getenv("WEAVIATE_API_KEY")

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={
            "X-OpenAI-Api-Key": os.environ["OPENAI_API_TOKEN"]
        }
    )

    print(client.is_ready())


    embedding_model = OpenAIEmbeddings(
        api_key=os.getenv("OPENAI_API_TOKEN")
    )
    vectorstore = WeaviateVectorStore(client=client, index_name="Triple", text_key="triple", embedding=embedding_model, attributes=["source"])

    retriever = vectorstore.as_retriever()
    llm = ChatOpenAI(
        temperature=0,
        api_key=os.getenv("OPENAI_API_TOKEN"),
        model_name="gpt-3.5-turbo")
    chain = RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
    )

    def prettychain(question: str) -> str:
        """Pretty print the chain's response to a question"""
        response = chain({"question": question},
            return_only_outputs=True,)
        print(textwrap.fill(response['answer'], 60))

    def prettychain3(question: str) -> None:
        """Pretty print the chain's response to a question, including sources"""
        response = chain({"question": question}, return_only_outputs=True)
        answer = response.get("answer", "No answer provided.")
        sources = response.get("sources", "No sources found.")
        print("Answer:")
        print(textwrap.fill(answer, 60))
        print("\nSources:")
        print(textwrap.fill(sources, 60))

    def prettychain2(question) -> str:
        """Pretty print the response to a question directly from the LLM"""
        response = llm.invoke(question)
        print(response.content)

    question = "What does the T1115 technique read?"
    print(question)
    print("-----------------------------------------RAG-------------------------------------------------")
    prettychain3(question)
    print("-----------------------------------------LLM-------------------------------------------------")
    prettychain2(question)
    print("--------------------------------------------------------------------------------------")

    question2 = "What is the definition of PacketLog?"
    print(question2)
    print("-----------------------------------------RAG-------------------------------------------------")
    prettychain3(question2)
    print("-----------------------------------------LLM-------------------------------------------------")
    prettychain2(question2)
    print("--------------------------------------------------------------------------------------")
    #print(vectorstore.similarity_search(question))
    #print(retriever.get_relevant_documents(question))



if __name__ == "__main__":
    rag()

True
What does the T1115 technique read?
-----------------------------------------RAG-------------------------------------------------
Answer:
The T1115 technique reads the clipboard.

Sources:
T1115--reads--Clipboard
-----------------------------------------LLM-------------------------------------------------
The T1115 technique, also known as "Clipboard Modification," reads data from the Windows clipboard. This technique is commonly used by attackers to steal sensitive information such as passwords, credit card numbers, or other confidential data that users may have copied to their clipboard.
--------------------------------------------------------------------------------------
What is the definition of PacketLog?
-----------------------------------------RAG-------------------------------------------------
Answer:
PacketLog is defined as a log of all the network packet data
captured from a network by a network sensor (i.e., packet
analyzer).

Sources:
PacketLog--definition--A log of 

In [11]:
import os
from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import weaviate
from weaviate.classes.init import Auth
import textwrap

def rag():
    weaviate_url = "https://ztzcuq1r6ogar9anhtr2a.c0.europe-west3.gcp.weaviate.cloud"
    weaviate_api_key = "40WeAARm6JgzWOYN9rGUiY2ANZOlG4PtFmiY"

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={
            "X-OpenAI-Api-Key": os.environ["OPENAI_API_TOKEN"]
        }
    )

    print(client.is_ready())


    embedding_model = OpenAIEmbeddings(
        api_key=os.getenv("OPENAI_API_TOKEN")
    )
    vectorstore = WeaviateVectorStore(client=client, index_name="Triple", text_key="triple", embedding=embedding_model, attributes=["source"])

    retriever = vectorstore.as_retriever()
    llm = ChatOpenAI(
        temperature=0,
        api_key=os.getenv("OPENAI_API_TOKEN"),
        model_name="gpt-3.5-turbo")
    chain = RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
    )

    def prettychain(question: str) -> str:
        """Pretty print the chain's response to a question"""
        response = chain({"question": question},
            return_only_outputs=True,)
        print(textwrap.fill(response['answer'], 60))

    #def prettychain(question) -> str:
    #    """Pretty print the chain's response to a question"""
    #    response = chain({"question": question}, return_only_outputs=True)
    #    print(response)

    def prettychain2(question) -> str:
        """Pretty print the response to a question directly from the LLM"""
        response = llm.invoke(question)
        print(response.content)

    question = "What does the user do in my architecture?"
    print(question)
    print("RAG")
    prettychain(question)
    print("LLM")
    prettychain2(question)
    print("--------------------------------------------------------------------------------------")
    question2 = "How many types of mail servers do I have in my architecture?"
    print(question2)
    print("RAG")
    prettychain(question2)
    print("LLM")
    prettychain2(question2)
    print("--------------------------------------------------------------------------------------")
    question3 = "Which mail server retrieves emails for the user?"
    print(question3)
    print("RAG")
    prettychain(question3)
    print("LLM")
    prettychain2(question3)


    #print(vectorstore.similarity_search(question))
    #print(retriever.get_relevant_documents(question))



if __name__ == "__main__":
    rag()

True
What does the user do in my architecture?
RAG
The user uses the EmailClient in the architecture.
LLM
In your architecture, the user interacts with the system by inputting commands, requests, or data through the user interface. The user's actions trigger processes within the system, such as data processing, communication with external systems, or generating outputs. The user may also receive feedback, notifications, or results from the system based on their interactions. Overall, the user plays a crucial role in driving the functionality and behavior of the architecture.
--------------------------------------------------------------------------------------
How many types of mail servers do I have in my architecture?
RAG
You have two types of mail servers in your architecture.
LLM
It is not possible to determine the number of mail servers in your architecture without more information about your specific setup. The number of mail servers can vary depending on the size and complexity 